In [759]:
import numpy as np
size = 100
x11 = np.random.uniform(low=0.0,high=1.0,size=size)
x12 = np.random.uniform(low=2.0,high=8.0,size=size)
x13 = np.random.uniform(low=6.0,high=8.0,size=size)

x21 = np.random.uniform(low=10.0,high=11.0,size=size)
x22 = np.random.uniform(low=12.0,high=18.0,size=size)
x23 = np.random.uniform(low=16.0,high=18.0,size=size)

x31 = np.random.uniform(low=20.0,high=21.0,size=size)
x32 = np.random.uniform(low=22.0,high=28.0,size=size)
x33 = np.random.uniform(low=26.0,high=28.0,size=size)

y1 = np.transpose(np.zeros(100))
y2 = np.transpose(np.ones(100))
y3 = np.transpose(np.ones(100) * 2)

X1 = np.transpose(np.array([x11, x12, x13]))
X2 = np.transpose(np.array([x21, x22, x23]))
X3 = np.transpose(np.array([x31, x32, x33]))
X = np.vstack((X1, X2, X3))

Y = np.hstack((y1, y2, y3))

print(X.shape)
print(Y.shape)

(300, 3)
(300,)


In [760]:
no_neurons_layer = [3,3]

def weight_init(input_matrix, no_neurons_layer):
    
    weight_matrix = list()
    no_features = input_matrix.shape[1]
    for neurons in no_neurons_layer:
        weight_matrix.append(np.random.rand(no_features + 1,neurons))
        no_features = neurons
    
    output_classes = len(np.unique(Y))
    weight_matrix.append(np.random.rand(no_features + 1, output_classes) )
    return weight_matrix

weight_matrix = weight_init(X, no_neurons_layer)   

In [761]:
layers = [None] * (len(no_neurons_layer) + 2)
learning_rate = 0.01

error = []
predicted = None
actual = None
grad_hiddenlayer = [None] * len(layers)
grad_activation = [None] * len(layers)
softmax_layer = len(layers) - 1

hidden_layers= [l for l in range(1,len(layers)-1)]

def add_bias(inp):
    return np.c_[inp,np.ones(inp.shape[0])]

def mat_multiply(mat1, mat2):
    return np.matmul(mat1, mat2)

def sigmoid(inp):
    return (1/(1 + np.exp(inp)))

def softmax(inp):
    op_exp = np.exp(inp)
    return op_exp/op_exp.sum(axis = 1)[:,None]

def output_class_encode(inp):
    inp = inp.astype(int)
    classes = np.unique(inp)
    no_classes = classes.shape[0]
    no_dim = inp.shape[0]
    zero_arr = np.zeros((no_dim, no_classes))
    zero_arr[np.arange(no_dim), inp] = 1.0
    return zero_arr

def least_squares_error(actual, predicted):
    return 0.5 * np.sum(np.square(np.subtract(predicted,actual)), axis = 0) * (1/actual.shape[0])

def cross_entropy(actual, predicted):
    return np.sum(np.multiply(actual, np.log(predicted)) * -1)
  
def derivative_loss_output_activation(): #d(Loss)/d(softmax)
    grad_softmax = (actual - predicted) * -1
    grad_activation[softmax_layer] = [None] * grad_softmax.shape[1]
    for i in range(grad_softmax.shape[1]):
        grad_activation[softmax_layer][i] = grad_softmax[:,i]

def derivative_sigmoid(): #d(sigmoid/d(activation_input)) or dh/da
    global grad_sigmoid
    
    grad_sigmoid = [None] * len(layers)  
    
    for i in range(1,len(layers)-1):
        grad_sigmoid[i] = [None] * len(layers[i])  
        for j in range(layers[i].shape[1]):
            grad_sigmoid[i][j] = layers[i][:,j] * (1 - layers[i][:,j])
        
def derivative_activation_previous_activation(): #d(Loss)/d(hidden_l)
           
    for i in sorted(hidden_layers,reverse = True):
        grad_hiddenlayer[i] = [None] * layers[i].shape[1]
        grad_activation[i] = [None] * layers[i].shape[1]
        for j in range(layers[i].shape[1]):
            grad_hiddenlayer[i][j] = np.zeros(layers[i].shape[0])
            for m in range(layers[i+1].shape[1] - 1):#-1 because bias doesn't connect to previous layer activation
                grad_hiddenlayer[i][j] += grad_activation[i+1][m] * np.transpose(weight_matrix[i])[m][j]
                
            grad_activation[i][j] = np.multiply(grad_hiddenlayer[i][j],grad_sigmoid[i][j])

def derivative_activation_weight():
    
    global grad_weights
    grad_weights = [None] * len(layers)
    
    for i in range(1, len(layers)):        
        if i == len(layers) -1:
            units = range(layers[i].shape[1]) #
            grad_weights[i] = [None] * (layers[i].shape[1])
        else:
            units = range(layers[i].shape[1] -1)
            grad_weights[i] = [None] * (layers[i].shape[1] - 1)
            
        for k in units:
            grad_weights[i][k] = [None] * layers[i-1].shape[1]
            for j in range(layers[i-1].shape[1]):
                grad_weights[i][k][j] = np.sum(np.multiply(layers[i-1][:,j], grad_activation[i][k]))/grad_activation[i][k].shape

def update_weight(learning_rate):
    for i in range(1,len(grad_weights)):
        weight_matrix[i-1] = (weight_matrix[i-1] - (learning_rate * np.array(np.transpose(grad_weights[i]))[0]))
    
def feed_forward(input_matrix, ouptut_class, no_neurons_layer):
    
    layers[0] = input_matrix #input layer

    for layer in range(len(no_neurons_layer) + 1):
        layers[layer] = add_bias(layers[layer])
        layers[layer + 1] = sigmoid(mat_multiply(layers[layer], weight_matrix[layer]))
    predicted_op = softmax(layers[len(no_neurons_layer)+1])
    actual_op = output_class_encode(ouptut_class)
    
    global predicted, actual
    predicted = predicted_op
    actual = actual_op
    
    error.append(cross_entropy(actual_op,predicted_op))

def backpropagation():
    derivative_loss_output_activation() #d(Loss)/d(softmax) or dL/dal
    derivative_sigmoid() #d(sigmoid/d(activation_input)) or dh/da
    
    derivative_activation_previous_activation() #d(Loss)/d(hidden_l) or dL/dhl
    derivative_activation_weight()
    update_weight(learning_rate)

feed_forward(X, Y, no_neurons_layer)    
derivative_loss_output_activation()

In [773]:
if 11001%1000 == 0:
    print('a')

In [748]:
for i in range(100000):
    feed_forward(X, Y, no_neurons_layer)    
    backpropagation()